# Overwatch Hero Survey (September 2019)

In [107]:
import pandas as pd
import seaborn as sb
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from matplotlib import pyplot as plt

# load flat csv files -- the survey results are a little clunky
hero_names_df = pd.read_csv("../data/overwatch_heroes.csv").loc[:,["Hero_name"]]
basic_info_df = pd.read_csv("../data/table_basic_info.csv")
hero_ratings_df = pd.read_csv("../data/table_hero_ratings.csv")

# join on Response_id such that we can more easily query the result set
joined_df = basic_info_df.set_index('Response_id').join(
    hero_ratings_df.set_index('Response_id'), 
    lsuffix='caller', 
    rsuffix='_other')

In [108]:
def hero_vs_fun_factor(df):
    return (df.query('Response_type == "Playing_vs"')
            .groupby('Hero')
            .mean())

## Q: Which heroes are the least fun to play against?

In [109]:

print(hero_vs_fun_factor(joined_df)
      .nsmallest(10, 'Value'))

               Value
Hero                
Doomfist    1.588629
Symmetra    1.693980
Mei         1.841973
Orisa       1.859532
Bastion     1.928930
Sombra      1.987458
Sigma       2.229933
Moira       2.306856
Widowmaker  2.350334
Reaper      2.484114


## Q: Which heroes are the most fun to play against?

In [110]:
print(hero_vs_fun_factor(joined_df)
      .nlargest(10, 'Value'))

                Value
Hero                 
Reinhardt    4.254181
Zenyatta     4.229933
Ana          4.127090
Soldier: 76  4.031773
Winston      3.989967
Zarya        3.834448
Lucio        3.794314
McCree       3.649666
D.va         3.628763
Ashe         3.525084


In [111]:
def hero_balance(df):
    return (joined_df.query('Response_type == "Balance"')
            .groupby('Hero')
            .mean()
            .sort_values(by='Value'))

## Q: Which heroes are perceived to be the most over-tuned?

In [112]:
print(hero_balance(joined_df)
      .nlargest(10, 'Value'))

               Value
Hero                
Orisa       4.262542
Sigma       4.260033
Doomfist    4.124582
Symmetra    3.796823
Moira       3.786789
Reaper      3.688963
Widowmaker  3.637960
Hanzo       3.623746
Mei         3.602843
Baptiste    3.565217


## Q: Which heroes are perceived to be the weakest?

In [113]:
print(hero_balance(joined_df)
      .nsmallest(10, 'Value'))

                Value
Hero                 
Soldier: 76  1.881271
Winston      2.058528
Reinhardt    2.296823
Brigitte     2.367057
Genji        2.385452
D.va         2.549331
Tracer       2.629599
Zenyatta     2.636288
Torbjorn     2.767559
Roadhog      2.782609


## Q: Which heroes are perceived to be well-balanced?

In [114]:
print(hero_balance(joined_df)
      .query('Value > 2.4 & Value < 3.6'))

                  Value
Hero                   
D.va           2.549331
Tracer         2.629599
Zenyatta       2.636288
Torbjorn       2.767559
Roadhog        2.782609
Mercy          2.837793
Ashe           2.894649
Pharah         2.913043
Junkrat        2.932274
Zarya          2.936455
Ana            2.981605
Wrecking Ball  3.035953
Lucio          3.071906
McCree         3.120401
Bastion        3.168060
Sombra         3.219900
Baptiste       3.565217


### Let's break down balance by hero & see the range of votes

In [115]:
@interact
def hero_balance_histograms(column=list(sorted(hero_names_df['Hero_name']))):
    hero_playing_vs_df = (joined_df.query('Response_type == "Playing_vs"')
                          .groupby('Hero'))['Value']

    sb.set_style("ticks")
    dp = sb.distplot(hero_playing_vs_df.get_group(column), bins=5, kde=False)
    dp.set(xlabel=f'Playing vs {column}', ylabel='Counts')
    plt.show()        

interactive(children=(Dropdown(description='column', options=('Ana', 'Ashe', 'Baptiste', 'Bastion', 'Brigitte'…

## Q: Which heroes are unpopular relative to their perceived balance level? 

In [116]:
def hero_fun_vs_balance(df):
    # Q: which heroes are unpopular relative to their perceived balance level? 
    # (i.e. people dislike playing against them even if they're not considered OP)
    playing_vs_df = (df.query('Response_type == "Playing_vs"')
     .groupby('Hero')
     .mean())    

    balance_df = (df.query('Response_type == "Balance"')
     .groupby('Hero') 
     .mean()
     .query('Value > 2.25 & Value < 3.75'))

    fun_balance_df = (playing_vs_df.join(
        balance_df,    
        lsuffix='_fun', 
        rsuffix='_balance')
      .query('Value_balance.notnull()')) # filter out the invalid joined data; should do this first though

    # this is pretty arbitrary, but gives us a decent idea of 'fun/unfun' vs balance
    fun_balance_ratio_df = fun_balance_df["Value_fun"] - fun_balance_df["Value_balance"]
    fun_balance_ratio_df.sort_values(inplace=True)
    
    return fun_balance_ratio_df

    # note: this would be better served as a chart (maybe divide by role) that shows 
    # the perceived fun value overlayed on the balance value.

In [117]:
print(hero_fun_vs_balance(joined_df))

Hero
Mei             -1.760870
Widowmaker      -1.287625
Bastion         -1.239130
Sombra          -1.232441
Reaper          -1.204849
Hanzo           -0.982441
Baptiste        -0.749164
Junkrat         -0.294314
Wrecking Ball   -0.166388
Torbjorn        -0.049331
Brigitte         0.255853
Pharah           0.278428
Roadhog          0.520903
McCree           0.529264
Mercy            0.619565
Ashe             0.630435
Lucio            0.722408
Tracer           0.765886
Zarya            0.897993
Genji            1.044314
D.va             1.079431
Ana              1.145485
Zenyatta         1.593645
Reinhardt        1.957358
dtype: float64


## Q: When playing as our main, which heroes are least fun to play against?

In [118]:
@interact
def hero_fun_vs_balance_by_main(column=list(sorted(hero_names_df['Hero_name']))):
    hero_name = column
    hero_playing_as_df = (joined_df.query(
        f'Hero_favourite == "{hero_name}" & Response_type == "Playing_vs"'))
    
    # for hero x, we have results for playing against all heroes. 
    # spin through every result in this set and query for the top 3 most-hated
    least_fun_group = (hero_playing_as_df
                       .query(f'Response_type == "Playing_vs"')
                       .groupby("Hero"))

    least_fun_mean = least_fun_group.mean()
    print(least_fun_mean.sort_values(by='Value')
          .nsmallest(10, 'Value'))
    
    # could also plot the histogram for least N fun heroes to play against when maining X
    #least_fun_mean.plot.hist()
    #dp = sb.distplot(least_fun_mean, bins=5, kde=False)
    #dp.set(xlabel=f'Playing as {hero_name}', ylabel='Count')    
    #plt.show()
    

interactive(children=(Dropdown(description='column', options=('Ana', 'Ashe', 'Baptiste', 'Bastion', 'Brigitte'…